## GLMAT: 2D kernel, song stimuli, ML+MC estimation

This notebook demonstrates the full assimilation technique using song stimuli. The song waveform is processed to a 2D spectrogram, then convolved with a 2D STRF to produce the "voltage" of the GLMAT model. The adaptation "current" is calculated by convolving the spike trains with two exponential kernels. The goal of the assimilation is to estimate the parameters of the RF and the adaptation kernels. The parameter count of the RF is minimized by using a low-rank approximation (i.e., an outer product of two vectors) and by projecting time into a basis set of raised cosine filters that are spaced exponentially.

The approach is to use elastic-net penalized maximum-likelihood estimation to get a first guess at the parameters. This model uses the time-compression basis set but does not factorize the RF. The next step involves factorizing the RF to further reduce the parameters and then using MCMC to sample the posterior distribution of the parameters.

In [ ]:
from __future__ import print_function, division
import os
import sys
import imp
import numpy as np

import mat_neuron._model as mat
from dstrf import strf, mle, io

# plotting packages
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.aspect'] = 'auto'
mpl.rcParams['image.cmap'] = 'jet'

The MAT model is governed by a small number of parameters: the spike threshold (omega), the amplitudes of the adaptation kernels (alpha_1, alpha_2), the time constants of the adaptation kernels (tau_1, tau_2), and the absolute refactory period. In addition, a function must be chosen for spike generation. The 'softplus' function, log(1 + exp(mu)), is a good choice because it doesn't saturate as readily when mu is large. Because there can only be one spike per bin, saturation causes the estimated parameters to be less than the true parameters.

In [ ]:
# model parameters: (ω, α1, α2, τ1, τ2, tref)
matparams = np.asarray([7, 100, 2, 10, 200, 2], dtype='d')
model_dt = 0.5
nlin = "exp"

matparams_i = [0,1,2]
matparams_n = len(matparams_i)
matparams_f = matparams[matparams_i]

In [ ]:
# STRF: keep this very simple for proof of principle
stim_dt = 3.0
upsample = int(stim_dt / model_dt)
spec_window = 4.0
spec_compress = 10
f_min = 1.0
f_max = 8.0
nfreq = 20
ntau  = 40
# these parameters influence how the STRF dimensional reduction will work
ntbas = 10       # number of temporal basis functions
krank = 2        # rank of bilinear approximation
kcosbas = strf.cosbasis(ntau, ntbas)
ntbas = kcosbas.shape[1]

Here we load some data from a real neural recording from the CRCNS dataset. We're going to replace the actual neural response with a simulation based on the dstrf model. In the original experiment, stimuli were presented individually in a pseudorandom order. To simplify the model, we concatenate the stimuli, setting padding between the stimuli sufficient to capture any offset responses.

The method of concatenation is a bit clumsy, because we don't know what was "presented" during the silent gaps. Simply filling in zeros is not a good idea, because this creates edges in the spectrogram that were not present in the actual stimulation. The current solution is to use the first and last frames of the spectrograms, but some sort of linear ramping to further eliminate transients would be even better.

In [ ]:
# data parameters
pad_before = 0    # how much to pad stimulus before onset
pad_after = ntau * stim_dt # how much to pad after offset
n_trials = 10     # set the number of trials to use (needs to be the same for all stimuli)
p_test = 0.2      # proportion of trials to use for test

# load stimuli and responses
root = os.path.join(os.environ["HOME"], "data", "crcns")
cell = "blabla0903_2_B"
stim_type = "conspecific"
data = io.load_crcns(cell, stim_type, root, spec_window, stim_dt, f_min=f_min, f_max=f_max, f_count=nfreq, 
                     compress=spec_compress, gammatone=True)
io.pad_stimuli(data, pad_before, pad_after, fill_value=0.0)
io.preprocess_spikes(data, model_dt, matparams[3:5])

n_test = int(p_test * len(data))
# split into assimilation and test sets and merge stimuli
assim_data = io.merge_data(data[:-n_test])
test_data = io.merge_data(data[-n_test:])

In [ ]:
plt.subplot(211).imshow(assim_data['stim'], extent=(0, assim_data["duration"], f_min, f_max))
ax = plt.subplot(212)
for i, d in enumerate(assim_data["spike_v"]):
    if i >= n_trials: break
    ax.vlines(d, i, i + 0.5)
for ax in plt.gcf().axes:
    ax.set_xlim(272, 900)
    # ax.set_xlim(0, assim_data["duration"])

In [ ]:
# calculate the binary spike vectors and convolve with spike-history kernels
assim_resp = []
stim = assim_data["stim"]
nframes = stim.shape[1]
nbins = nframes * upsample
for i, spk in enumerate(assim_data["spikes"]):
    if i >= n_trials: break
    spike_t = (spk / model_dt).astype('i')
    spike_v = np.zeros(nbins, dtype='i')
    spike_v[spike_t] = 1
    d = {"spike_t": spike_t,
         "spike_v": spike_v,
         "H": mat.adaptation(spike_v, matparams[3:5], model_dt)}
    assim_resp.append(d)

In [ ]:
stim = assim_data["stim"]
plt.subplot(311).imshow(assim_data["stim"], extent=(0, assim_data["duration"], f_min, f_max))
t_spike = np.linspace(0, assim_data["duration"], assim_data["spike_v"].shape[0])
plt.subplot(312).plot(t_spike, assim_data["spike_v"].sum(1))
ax = plt.subplot(313)
for i, spk in enumerate(assim_data["spike_t"]):
    ax.vlines(spk * model_dt, i, i + 0.5)
    ax.plot(t_spike, assim_data["spike_h"][:, i, :])
for ax in plt.gcf().axes:
    ax.set_xlim(1750, 2200)

## Estimate parameters

The reg_alpha and reg_lambda parameters set the L1 and L2 penalties for the initial ML estimation. Note that we supply the nonlinearity function to the constructor too, as this determines how the log-likelihood is calculated.

In [ ]:
# initial guess of parameters using penalized ML. Note that we provide the cosine basis set to the constructor of
# mle.estimator, which causes the design matrix to be in the cosine basis set
spikes = np.stack([d["spike_v"] for d in assim_resp], axis=1)
mlest = mle.estimator(stim, spikes, kcosbas, matparams[3:5], stim_dt, model_dt, nlin="exp")
%time w0 = mlest.estimate(reg_lambda=0, reg_alpha=1e1)
# TODO: cross-validation to find best regularization parameters

In [ ]:
print(w0[:3])
# the RF is in the cosine basis set, which we need to convert back in order to visualize
rf_sta = strf.as_matrix(mlest.sta(), kcosbas)
rf_mle = strf.as_matrix(w0[3:], kcosbas)
k0f, k0t = strf.factorize(strf.as_matrix(w0[3:], ntbas), krank)
rf_mlb = strf.from_basis(np.dot(k0f, k0t), kcosbas)
plt.subplot(221).imshow(rf_sta, extent=(0, ntau, f_min, f_max))
plt.title("STA")
plt.subplot(222).imshow(rf_mle, extent=(0, ntau, f_min, f_max))
plt.title("MAP estimate")
plt.subplot(224).imshow(rf_mlb, extent=(0, ntau, f_min, f_max))
plt.title("MAP (rank {})".format(krank))

We'll use the ML estimate to seed the MCMC sampler. We're going to reduce the size of the parameter space by factorizing the RF (i.e., a bilinear approximation). Note that we try to use the mlest object as much as possible to do the calculations rather than reimplement things; however, there can be some significant performance enhancements from an optimized implementation.

In [ ]:
# estimate parameters using emcee
from neurofit import priors, costs, utils, startpos

# the MAT parameters are just bounded between reasonable limits. These may need to be expanded when using real data.
mat_prior = priors.joint_independent(
                [ priors.uniform( 0,  20),
                  priors.uniform(-50,  200),
                  priors.uniform(-5,   10),
                ])

# lasso prior on RF parameters. This is the same as the reg_alpha parameter in the penalized MLE
rf_alpha = 10.


if nlin == "exp":
    def loglike(V, H, spike_t, alpha, dt):
        mu = V - np.dot(H, alpha)
        return mu[spike_t].sum() - np.exp(mu).sum() * dt
elif nlin == "softplus":
    def loglike(V, H, spike_t, alpha, dt):
        mu = V - np.dot(H, alpha)
        # don't use np.log1p(np.exp(mu)) - it overflows!
        lmb = mat.softplus(mu)
        return np.log(lmb[spike_t]).sum() - lmb.sum() * dt 

def lnpost(theta):
    """Posterior probability for dynamical parameters"""
    mparams = theta[:3]
    rfparams = theta[3:]
    rf_prior = -np.sum(np.abs(rfparams)) * rf_alpha
    ll = mat_prior(mparams) + rf_prior
    if not np.isfinite(ll):
        return -np.inf
    # reassemble strf from low-rank factors
    k = strf.defactorize(rfparams, nfreq, krank).flatten()
    w = np.r_[mparams, k]
    # here we do some optimization. We'll use mlest object to calculate the interpolated convolution of the 
    # RF with the stimulus, but then directly calculate the loglikelihood ourselves on each trial. This can
    # produce some significant savings on multi-trial responses.
    Vi = mlest.V_interp(w).squeeze() - mparams[0]
    lp = 0
    for d in assim_resp:
        lp += loglike(Vi, d["H"], d["spike_t"], mparams[1:3], model_dt)
    return lp


In [ ]:
# get low-rank approx of the ML STRF
w0_bl = np.r_[w0[:3], k0f.flatten(), k0t.flatten()]
print("lnpost of ML estimate: {}".format(lnpost(w0_bl)))
%timeit lnpost(w0_bl)

This code starts the MCMC sampler. We initialize the walkers (chains) in a gaussian around the ML estimate, with standard deviation 2x the absolute value of the best guess. The model converges fairly quickly, but then we let it sample for a while.

In [ ]:
import emcee
# assimilation parameters
if sys.platform == 'darwin':
    nthreads = 1
else:
    nthreads = 8
nwalkers = 500
nsteps = 3000

# initialize walkers
pos = p0 = startpos.normal_independent(nwalkers, w0_bl, np.abs(w0_bl) * 2)
# initialize the sampler
sampler = emcee.EnsembleSampler(nwalkers, w0_bl.size, lnpost, threads=nthreads)

In [ ]:
# start the sampler
tracker = utils.convergence_tracker(nsteps, 100)
for pos, prob, like in tracker(sampler.sample(pos, iterations=nsteps, storechain=True)): 
    continue

In [ ]:
print("lnpost of p median: {}".format(np.median(prob)))
print("average acceptance fraction: {}".format(sampler.acceptance_fraction.mean()))
try:
    print("autocorrelation time: {}".format(sampler.acor))
except:
    pass    
w1 = np.median(pos, 0)
rfparams = w1[3:]
rf_map = strf.from_basis(strf.defactorize(rfparams, nfreq, krank), kcosbas)
print(w0[:matparams_n])
print(w1[:matparams_n])
plt.subplot(222)
plt.imshow(rf_mle, extent=(0, ntau, f_min, f_max))
plt.subplot(223)
plt.imshow(rf_mlb, extent=(0, ntau, f_min, f_max))
plt.subplot(224)
plt.imshow(rf_map, extent=(0, ntau, f_min, f_max))

In [ ]:
from corner import corner
sns.set_style("whitegrid")

mpos = pos[:,:matparams_n]
matlabs = ['w','a1','a2',]
c = corner(mpos,
       bins=50, smooth=2,smooth1d=0,
       labels=matlabs,
       truths=matparams[:3])

In [ ]:
def predict_spikes(V, params, dt, upsample):
    omega, a1, a2, t1, t2, tref = params
    if nlin == "exp":
        f = mat.predict_poisson
    elif nlin == "softplus":
        f = mat.predict_softplus
    return f(V - omega, (a1, a2), (t1, t2), tref, dt, upsample)

# posterior predictive distribution
stim = test_data["stim"]
t_stim = np.linspace(0, test_data["duration"], stim.shape[1])
plt.subplot(311).imshow(stim, extent=(0, test_data["duration"], f_min, f_max))
vax = plt.subplot(312)
spax = plt.subplot(313)
for j, spk in enumerate(test_data["spikes"]):
    spax.vlines(spk, j, j + 0.5, 'r')
mparams = matparams.copy()
samples = np.random.permutation(nwalkers)[:j]
for i, idx in enumerate(samples):
    rfparams = pos[idx, matparams_n:]
    k = strf.defactorize(rfparams, nfreq, krank)
    rf = strf.from_basis(k, kcosbas)
    V = strf.convolve(stim, rf)
    vax.plot(t_stim, V)
    mparams[matparams_i] = pos[idx, :matparams_n]
    S = predict_spikes(V, mparams, model_dt, upsample)
    spk_t = S.nonzero()[0]
    spax.vlines(spk_t * model_dt, i + j + 1, i + j + 1.5)

for ax in plt.gcf().axes:
    ax.set_xlim(0, 1000)